In [4]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import time
from time import time_ns
%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(1)
seed(1)
scala=1

In [5]:
def readTxtToData(data):
    columns = ['ntp_time','data_collection_count','block_sequence_length','throughput','num_devices','lower_outlier']
    row = {
        columns[0]: [],
        columns[1]: [],
        columns[2]: [],
        columns[3]: [],
        columns[4]: [],
        columns[5]: [],
    }
    
    start1 = time.time()
    train_start = time_ns()
    #READ FILE A FILE
    a_file = open(data, "r")
    lines = a_file.readlines()
    last_xlines = lines[-(9999):]
    a_file.close()
    #CLOSE FILE
    baris = 0
    for data in last_xlines:
        split = data.split(";")
        # row['mac'].append("08:3A:F2:A9:8D:85")
        start = float(split[1]) + (float(split[2])/1000000)
        row['ntp_time'].append(str(start))
        baris = baris + 1
        row['data_collection_count'].append(baris)
        row['block_sequence_length'].append(1)
        end = float(split[3]) + (float(split[4])/1000000)
        row['throughput'].append(round(float(split[0])/(end-start))/scala )
        row['num_devices'].append(1)
        row['lower_outlier'].append(False)
            
    end1 = time.time()
    train_end = time_ns()
    print(end1-start1)  
    df = pd.DataFrame(row, columns=columns)
    print (df)
    return df

In [6]:
train = readTxtToData('../cnn/10k.txt')
test = readTxtToData('../cnn/test.txt')

0.08417153358459473
               ntp_time  data_collection_count  block_sequence_length  \
0     1664599947.122157                      1                      1   
1      1664599947.73606                      2                      1   
2     1664599948.370683                      3                      1   
3     1664599948.977481                      4                      1   
4     1664599949.691041                      5                      1   
...                 ...                    ...                    ...   
9994  1664615714.378134                   9995                      1   
9995  1664615715.090597                   9996                      1   
9996  1664615716.124379                   9997                      1   
9997  1664615722.207594                   9998                      1   
9998   1664615722.98786                   9999                      1   

      throughput  num_devices  lower_outlier  
0        57059.0            1          False  
1        